Required initial steps:
    
    * The text file from https://afltables.com/afl/stats/biglists/bg3.txt is saved as llist.txt
    * The file is then processed through the bash script file a_updater.sh to convert it to a csv and make other modifications.
    * The bash file saves the processed file as newlist.csv, ready to be imported from within Python

In [1]:
import pandas as pd
cols = ['seq','date','round','team1','gls_1','bhs_1','pts_1','team2','gls_2','bhs_2','pts_2','venue']

In [2]:
upd_round = pd.read_csv("/Users/stevegabriel/data_projects/sports_data/newlist.csv", names = cols)
trans = upd_round

In [3]:
# Modify index to start at 1 
trans.index += 1
trans.tail()

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,pts_2,venue
15322,15322,30-Jun-2018,R15,Gold Coast,8,12,60,Collingwood,14,15,99,Carrara
15323,15323,30-Jun-2018,R15,GW Sydney,13,17,95,Hawthorn,13,6,84,Sydney Showground
15324,15324,1-Jul-2018,R15,Melbourne,18,9,117,St Kilda,18,11,119,M.C.G.
15325,15325,1-Jul-2018,R15,Essendon,19,11,125,North Melbourne,16,12,108,Docklands
15326,15326,1-Jul-2018,R15,Fremantle,9,10,64,Brisbane Lions,18,11,119,Perth Stadium


In [4]:
# Add states
nsw = ['S.C.G.','Sydney Showground','Stadium Australia','Blacktown','Albury']
qld = ['Gabba','Carrara','Cazaly\'s Stadium','Brisbane Exhibition']
sa = ['Football Park','Adelaide Oval']
wa = ['W.A.C.A.','Perth Stadium','Subiaco']
nt = ['Marrara Oval','Traeger Park']
tas = ['York Park','Bellerive Oval','North Hobart']
act = ['Manuka Oval','Bruce Stadium']
intl = ['Wellington','Jiangwan Stadium']

In [5]:
def locations(x):
    if x in nsw:
        return "NSW"
    elif x in sa:
        return "SA"
    elif x in wa:
        return "WA"
    elif x in nt:
        return "NT"
    elif x in tas:
        return "TAS"
    elif x in intl:
        return "INTL"
    elif x in act:
        return "ACT"
    elif x in qld:
        return "QLD"
    else: 
        return "VIC"

In [6]:
trans["state"] = trans["venue"].apply(locations)
trans["state"].value_counts()

VIC     12995
WA        632
SA        565
NSW       504
QLD       466
TAS        89
ACT        47
NT         23
INTL        5
Name: state, dtype: int64

In [7]:
## 1. HVAR & AGGR
trans['hvar'] = trans['pts_1'] - trans['pts_2']
trans['aggr'] = trans['pts_1'] + trans['pts_2']

In [8]:
## 2. YEAR & MONTH
import time
import datetime

In [9]:
trans['date'] = pd.to_datetime(trans['date'])
trans['year'], trans['month'] = trans['date'].dt.year, trans['date'].dt.month

In [10]:
## 3. ABBREVIATED CLUB NAMES
adelaide = ['Adelaide']
brisbane = ['Brisbane Bears','Brisbane Lions']
carlton = ['Carlton']
collingwood = ['Collingwood']
essendon = ['Essendon']
fitzroy = ['Fitzroy']
footscray = ['Footscray']
fremantle = ['Fremantle']
geelong = ['Geelong']
gold_coast = ['Gold Coast']
gws = ['GW Sydney']
hawthorn = ['Hawthorn']
melbourne = ['Melbourne']
north_melbourne = ['North Melbourne','Kangaroos']
port_adelaide = ['Port Adelaide']
richmond = ['Richmond']
south_melbourne = ['South Melbourne']
st_kilda = ['St Kilda']
sydney = ['Sydney']
west_coast = ['West Coast']
western_bulldogs = ['Western Bulldogs', 'Footscray']
university = ['University']

In [11]:
def team_shorten(x):
    if x in adelaide:
        return "AD"
    elif x in brisbane:
        return "BR"
    elif x in carlton:
        return "CA"
    elif x in collingwood:
        return "CO"
    elif x in essendon:
        return "ES"
    elif x in fitzroy:
        return "FI"
    elif x in fremantle:
        return "FR"
    elif x in geelong:
        return "GE"
    elif x in gold_coast:
        return "GC"
    elif x in gws:
        return "GW"
    elif x in hawthorn:
        return "HA"
    elif x in melbourne:
        return "ME"
    elif x in north_melbourne:
        return "NM"
    elif x in port_adelaide:
        return "PA"
    elif x in richmond:
        return "RI"
    elif x in south_melbourne:
        return "SM"
    elif x in st_kilda:
        return "SK"
    elif x in sydney:
        return "SY"
    elif x in west_coast:
        return "WC"
    elif x in western_bulldogs:
        return "WB"
    elif x in university:
        return "UN"
    else: 
        return "NOT_FOUND"

In [12]:
trans["t1"] = trans["team1"].apply(team_shorten)
trans["t2"] = trans["team2"].apply(team_shorten)

In [13]:
trans['t1'].value_counts().sum()

15326

In [14]:
# 4. FIX TEAM NAMES ANOMOLIES

* Merge Footscray & Western Bulldogs as **Western Bulldogs**
* Merge North Melbourne with Kangaroos as **North Melbourne**
* Merge Brisbane Lions with Brisbane Bears as **Brisbane**
* Leave Sydney and **South Melbourne** as separate
* Leave **Fitzroy** as separate
* Modify GW Sydney to **GWS Giants**

In [15]:
# Function from Analytics Vidhya
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded

In [16]:
trans["team1"] = coding(trans["team1"], {'Footscray':'Western Bulldogs'})
trans["team2"] = coding(trans["team2"], {'Footscray':'Western Bulldogs'})
trans["team1"] = coding(trans["team1"], {'Kangaroos':'North Melbourne'})
trans["team2"] = coding(trans["team2"], {'Kangaroos':'North Melbourne'})
trans["team1"] = coding(trans["team1"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team2"] = coding(trans["team2"], {'Brisbane Bears':'Brisbane','Brisbane Lions':'Brisbane'})
trans["team1"] = coding(trans["team1"], {'GW Sydney':'GWS Giants'})
trans["team2"] = coding(trans["team2"], {'GW Sydney':'GWS Giants'})

In [17]:
trans["team2"].value_counts()

Collingwood         1269
Carlton             1247
Essendon            1218
Geelong             1214
St Kilda            1187
Melbourne           1187
Richmond            1115
North Melbourne      982
Hawthorn             973
Western Bulldogs     969
Fitzroy              960
South Melbourne      788
Sydney               427
West Coast           372
Brisbane             355
Adelaide             321
Fremantle            269
Port Adelaide        250
Gold Coast            84
GWS Giants            76
University            63
Name: team2, dtype: int64

In [18]:
# 5. ALL DRAWS (BOOLEAN)

In [19]:
def reg_draw(hvar):
    if hvar == 0:
        return 1
    else: return 0

In [20]:
trans["draw_rt"] = trans["hvar"].apply(reg_draw)

In [21]:
trans['draw_rt'].value_counts()

0    15168
1      158
Name: draw_rt, dtype: int64

### The 158 draws above, represent the VFL/AFL games that have concluded in a draw result

### The two ET (extra-time) draws below are statistically relevent, since they also ended regular full-time with both teams on the same score. However extra-time was applied to create a result, due to finals constraints.

In [22]:
def et_draw(seq):
    if seq == 10794:
        return 1
    elif seq == 13203:
        return 1
    else: return 0

In [23]:
trans["etime_draw"] = trans["seq"].apply(et_draw)
trans['etime_draw'].value_counts()

0    15324
1        2
Name: etime_draw, dtype: int64

In [24]:
### Add the two columns
trans['draws_all']  = trans['draw_rt'] + trans['etime_draw']
trans['draws_all'].value_counts()

0    15166
1      160
Name: draws_all, dtype: int64

In [25]:
# Check
trans.iloc[:,:][(trans.etime_draw == 1)]

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,state,hvar,aggr,year,month,t1,t2,draw_rt,etime_draw,draws_all
10794,10794,1994-09-10,QF,North Melbourne,15,24,114,Hawthorn,13,13,...,VIC,23,205,1994,9,NM,HA,0,1,1
13203,13203,2007-09-14,SF,Collingwood,13,15,93,West Coast,10,14,...,WA,19,167,2007,9,CO,WC,0,1,1


In [26]:
# 6. GAME CATEGORY
finals = ['EF','QF','SF','PF','GF']

In [27]:
def game_cat(x):
    if x in finals:
        return "final"
    else: 
        return "reg_season"

In [28]:
trans["type"] = trans["round"].apply(game_cat)

In [29]:
trans['type'].value_counts()

reg_season    14680
final           646
Name: type, dtype: int64

### So, there have been 646 finals up to the end of the 2017 season.

In [30]:
def grouping_2K(seq):
    n=500
    if seq <= 4*n:
        return 1
    elif seq <= 8*n:
        return 2
    elif seq <= 12*n:
        return 3
    elif seq <= 16*n:
        return 4
    elif seq <= 20*n:
        return 5
    elif seq <= 24*n:
        return 6
    elif seq <= 28*n:
        return 7
    else: return 8

In [31]:
trans["grp_2K"] = trans["seq"].apply(grouping_2K)

In [32]:
def grouping_1K(seq):
    n=1000
    if seq <= n:
        return 1
    elif seq <= 2*n:
        return 2
    elif seq <= 3*n:
        return 3
    elif seq <= 4*n:
        return 4
    elif seq <= 5*n:
        return 5
    elif seq <= 6*n:
        return 6
    elif seq <= 7*n:
        return 7
    elif seq <= 8*n:
        return 8
    elif seq <= 9*n:
        return 9
    elif seq <= 10*n:
        return 10
    elif seq <= 11*n:
        return 11
    elif seq <= 12*n:
        return 12
    elif seq <= 13*n:
        return 13
    elif seq <= 14*n:
        return 14
    elif seq <= 15*n:
        return 15
    else: return 16

In [33]:
trans["grp_1K"] = trans["seq"].apply(grouping_1K)

In [34]:
# Export the data frame for use in afl draws EDA
trans.to_csv('all_afl_wide.csv')

## 8. ADD res1 and res2 columns as categorical 1 = win, 2 = loss, 3 = draw based on hvar (ignoring the two extra time finals)

In [35]:
def result_cat_h(hvar):
    if hvar > 0:
        return 1
    elif hvar < 0:
        return 2
    else: return 3

In [36]:
def result_cat_a(hvar):
    if hvar > 0:
        return 2
    elif hvar < 0:
        return 1
    else: return 3

In [37]:
trans['res1'] = trans['hvar'].apply(result_cat_h)
trans['res2'] = trans['hvar'].apply(result_cat_a)

In [38]:
trans.tail(9)

,seq,date,round,team1,gls_1,bhs_1,pts_1,team2,gls_2,bhs_2,...,t1,t2,draw_rt,etime_draw,draws_all,type,grp_2K,grp_1K,res1,res2
15318,15318,2018-06-28,R15,Richmond,14,9,93,Sydney,11,1,...,RI,SY,0,0,0,reg_season,8,16,1,2
15319,15319,2018-06-29,R15,Western Bulldogs,16,7,103,Geelong,15,11,...,WB,GE,0,0,0,reg_season,8,16,1,2
15320,15320,2018-06-30,R15,Carlton,10,9,69,Port Adelaide,13,12,...,CA,PA,0,0,0,reg_season,8,16,2,1
15321,15321,2018-06-30,R15,Adelaide,12,16,88,West Coast,12,6,...,AD,WC,0,0,0,reg_season,8,16,1,2
15322,15322,2018-06-30,R15,Gold Coast,8,12,60,Collingwood,14,15,...,GC,CO,0,0,0,reg_season,8,16,2,1
15323,15323,2018-06-30,R15,GWS Giants,13,17,95,Hawthorn,13,6,...,GW,HA,0,0,0,reg_season,8,16,1,2
15324,15324,2018-07-01,R15,Melbourne,18,9,117,St Kilda,18,11,...,ME,SK,0,0,0,reg_season,8,16,2,1
15325,15325,2018-07-01,R15,Essendon,19,11,125,North Melbourne,16,12,...,ES,NM,0,0,0,reg_season,8,16,1,2
15326,15326,2018-07-01,R15,Fremantle,9,10,64,Brisbane,18,11,...,FR,BR,0,0,0,reg_season,8,16,2,1


## STAGE TWO
## 9. Reduce and re-order the data frame

In [39]:
tracker = trans
tracker = tracker[['year', 'team1','pts_1','team2','pts_2','venue','t1','t2','hvar']]
tracker.tail()

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar
15322,2018,Gold Coast,60,Collingwood,99,Carrara,GC,CO,-39
15323,2018,GWS Giants,95,Hawthorn,84,Sydney Showground,GW,HA,11
15324,2018,Melbourne,117,St Kilda,119,M.C.G.,ME,SK,-2
15325,2018,Essendon,125,North Melbourne,108,Docklands,ES,NM,17
15326,2018,Fremantle,64,Brisbane,119,Perth Stadium,FR,BR,-55


In [40]:
### Create two new columns with values 1 and 0 for the two subsets

In [41]:
tracker['subset_1'] = 1
tracker['subset_2'] = 0

/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [42]:
tracker.tail(9)

,year,team1,pts_1,team2,pts_2,venue,t1,t2,hvar,subset_1,subset_2
15318,2018,Richmond,93,Sydney,67,Docklands,RI,SY,26,1,0
15319,2018,Western Bulldogs,103,Geelong,101,Docklands,WB,GE,2,1,0
15320,2018,Carlton,69,Port Adelaide,90,M.C.G.,CA,PA,-21,1,0
15321,2018,Adelaide,88,West Coast,78,Adelaide Oval,AD,WC,10,1,0
15322,2018,Gold Coast,60,Collingwood,99,Carrara,GC,CO,-39,1,0
15323,2018,GWS Giants,95,Hawthorn,84,Sydney Showground,GW,HA,11,1,0
15324,2018,Melbourne,117,St Kilda,119,M.C.G.,ME,SK,-2,1,0
15325,2018,Essendon,125,North Melbourne,108,Docklands,ES,NM,17,1,0
15326,2018,Fremantle,64,Brisbane,119,Perth Stadium,FR,BR,-55,1,0


In [43]:
droplist_1 = ['subset_2']
droplist_2 = ['subset_1']

In [44]:
left_team = tracker.drop(droplist_1, axis = 1)
right_team = tracker.drop(droplist_2, axis = 1)

In [45]:
left_team.shape

(15326, 10)

In [46]:
right_team.shape

(15326, 10)

In [47]:
# Rename columns in each dframe 

In [48]:
left_team.columns = ['year','team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']
right_team.columns = ['year','opp_team','op_pts','team','tm_pts','venue','op','tm','hvar','lr']

### Now re-order the second dframe to match columns

### Note double brackets to re-order

In [49]:
right_team = right_team[['year', 'team','tm_pts','opp_team','op_pts','venue','tm','op','hvar','lr']]
right_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15322,2018,Collingwood,99,Gold Coast,60,Carrara,CO,GC,-39,0
15323,2018,Hawthorn,84,GWS Giants,95,Sydney Showground,HA,GW,11,0
15324,2018,St Kilda,119,Melbourne,117,M.C.G.,SK,ME,-2,0
15325,2018,North Melbourne,108,Essendon,125,Docklands,NM,ES,17,0
15326,2018,Brisbane,119,Fremantle,64,Perth Stadium,BR,FR,-55,0


In [50]:
right_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15322,2018,Collingwood,99,Gold Coast,60,Carrara,CO,GC,-39,0
15323,2018,Hawthorn,84,GWS Giants,95,Sydney Showground,HA,GW,11,0
15324,2018,St Kilda,119,Melbourne,117,M.C.G.,SK,ME,-2,0
15325,2018,North Melbourne,108,Essendon,125,Docklands,NM,ES,17,0
15326,2018,Brisbane,119,Fremantle,64,Perth Stadium,BR,FR,-55,0


In [51]:
left_team.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
15322,2018,Gold Coast,60,Collingwood,99,Carrara,GC,CO,-39,1
15323,2018,GWS Giants,95,Hawthorn,84,Sydney Showground,GW,HA,11,1
15324,2018,Melbourne,117,St Kilda,119,M.C.G.,ME,SK,-2,1
15325,2018,Essendon,125,North Melbourne,108,Docklands,ES,NM,17,1
15326,2018,Fremantle,64,Brisbane,119,Perth Stadium,FR,BR,-55,1


## CONCATENATION

### Now very easily, can concatenate the two dframes, as in si_week11

In [52]:
merge_ft = pd.concat([left_team, right_team], axis=0)
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1


#### The new long data frame consisting of 30,000 plus rows:

In [53]:
merge_ft.shape

(30652, 10)

In [54]:
# The following is it keep each game together (two rows per game)

In [55]:
merge_ft.sort_index(inplace=True)
merge_ft.head(10)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1
3,1897,Essendon,47,Geelong,24,Corio Oval,ES,GE,-23,0
4,1897,South Melbourne,27,Melbourne,44,Lake Oval,SM,ME,-17,1
4,1897,Melbourne,44,South Melbourne,27,Lake Oval,ME,SM,-17,0
5,1897,South Melbourne,40,Carlton,36,Lake Oval,SM,CA,4,1
5,1897,Carlton,36,South Melbourne,40,Lake Oval,CA,SM,4,0


In [56]:
## Create team result column

In [57]:
def neg_hvar(hvar):
        return -1*hvar

In [58]:
merge_ft["neg_dummy"] = merge_ft['hvar'].apply(neg_hvar)

In [59]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy
15324,2018,Melbourne,117,St Kilda,119,M.C.G.,ME,SK,-2,1,2
15325,2018,North Melbourne,108,Essendon,125,Docklands,NM,ES,17,0,-17
15325,2018,Essendon,125,North Melbourne,108,Docklands,ES,NM,17,1,-17
15326,2018,Fremantle,64,Brisbane,119,Perth Stadium,FR,BR,-55,1,55
15326,2018,Brisbane,119,Fremantle,64,Perth Stadium,BR,FR,-55,0,55


In [60]:
def team_var(hvar):
    if merge_ft['lr'] == 1:
        return hvar
    else: return neg_dummy

In [61]:
def new_lr(lr):
    if lr == 0:
        return -1
    else: return 1

In [62]:
merge_ft["new_lr"] = merge_ft['lr'].apply(new_lr)
merge_ft["tm_var"] = merge_ft['new_lr']*merge_ft['hvar']

In [63]:
merge_ft.tail()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,neg_dummy,new_lr,tm_var
15324,2018,Melbourne,117,St Kilda,119,M.C.G.,ME,SK,-2,1,2,1,-2
15325,2018,North Melbourne,108,Essendon,125,Docklands,NM,ES,17,0,-17,-1,-17
15325,2018,Essendon,125,North Melbourne,108,Docklands,ES,NM,17,1,-17,1,17
15326,2018,Fremantle,64,Brisbane,119,Perth Stadium,FR,BR,-55,1,55,1,-55
15326,2018,Brisbane,119,Fremantle,64,Perth Stadium,BR,FR,-55,0,55,-1,55


In [64]:
### Can now remove the columns new_lr and neg_dummy, which were used to create tm_var

In [65]:
merge_ft = merge_ft.drop(['neg_dummy', 'new_lr'], axis=1)

In [66]:
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,33
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0,-33
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,25
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0,-25
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,-23


In [67]:
# Note that the column 'lr' denotes left right, so there are only two different values.

In [68]:
merge_ft['lr'].value_counts()

1    15326
0    15326
Name: lr, dtype: int64

## New function for win/loss/draw (W/L/D)


In [69]:
def outcome(tm_var):
    if tm_var >= 1:
        return "W"
    elif tm_var <= -1:
        return "L"
    elif tm_var == 0:
        return "D"
    else: return "Unknown"

In [70]:
merge_ft["outcome"] = merge_ft['tm_var'].apply(outcome)

In [71]:
merge_ft.head()

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
1,1897,Fitzroy,49,Carlton,16,Brunswick St,FI,CA,33,1,33,W
1,1897,Carlton,16,Fitzroy,49,Brunswick St,CA,FI,33,0,-33,L
2,1897,Collingwood,41,St Kilda,16,Victoria Park,CO,SK,25,1,25,W
2,1897,St Kilda,16,Collingwood,41,Victoria Park,SK,CO,25,0,-25,L
3,1897,Geelong,24,Essendon,47,Corio Oval,GE,ES,-23,1,-23,L


## Example filtering view

In [72]:
# Carlton's last 20 games at the Sydney Cricket Ground

In [73]:
merge_ft.iloc[:,:][(merge_ft.tm == 'CA') & (merge_ft.venue == 'S.C.G.')].tail(20)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
9672,1988,Carlton,166,Sydney,80,S.C.G.,CA,SY,-86,0,86,W
9877,1989,Carlton,85,Sydney,57,S.C.G.,CA,SY,-28,0,28,W
10064,1990,Carlton,162,Sydney,90,S.C.G.,CA,SY,-72,0,72,W
10215,1991,Carlton,110,Sydney,124,S.C.G.,CA,SY,14,0,-14,L
10436,1992,Carlton,139,Sydney,130,S.C.G.,CA,SY,-9,0,9,W
10518,1993,Carlton,139,Sydney,95,S.C.G.,CA,SY,-44,0,44,W
10770,1994,Carlton,62,Sydney,70,S.C.G.,CA,SY,8,0,-8,L
10859,1995,Carlton,60,Sydney,132,S.C.G.,CA,SY,72,0,-72,L
11219,1997,Carlton,83,Sydney,142,S.C.G.,CA,SY,59,0,-59,L
12021,2001,Carlton,81,Sydney,91,S.C.G.,CA,SY,10,0,-10,L


In [72]:
# Carlton's last 10 games against Richmond

In [74]:
merge_ft.iloc[:,:][(merge_ft.tm == 'CA') & (merge_ft.op == 'RI')].tail(10)

,year,team,tm_pts,opp_team,op_pts,venue,tm,op,hvar,lr,tm_var,outcome
14169,2013,Carlton,101,Richmond,106,M.C.G.,CA,RI,-5,1,-5,L
14339,2013,Carlton,106,Richmond,96,M.C.G.,CA,RI,-10,0,10,W
14368,2013,Carlton,116,Richmond,96,M.C.G.,CA,RI,20,1,20,W
14383,2014,Carlton,86,Richmond,98,M.C.G.,CA,RI,12,0,-12,L
14581,2015,Carlton,78,Richmond,105,M.C.G.,CA,RI,-27,1,-27,L
14698,2015,Carlton,41,Richmond,71,M.C.G.,CA,RI,30,0,-30,L
14787,2016,Carlton,83,Richmond,92,M.C.G.,CA,RI,9,0,-9,L
14994,2017,Carlton,89,Richmond,132,M.C.G.,CA,RI,-43,1,-43,L
15109,2017,Carlton,58,Richmond,84,M.C.G.,CA,RI,26,0,-26,L
15201,2018,Carlton,95,Richmond,121,M.C.G.,CA,RI,26,0,-26,L
